<a href="https://colab.research.google.com/github/lalesafarzade/Recommendation_system_Project/blob/lale/4_Recommender_engine_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install scikit-surprise
import numpy as np
import pandas as pd
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader,SVD
from surprise.model_selection import cross_validate
from collections import defaultdict
from operator import itemgetter
import heapq

import os
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 6.8 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633721 sha256=336629683962e30d84752c84c236f54d4806c1c4843b62392f29a8beff8ec36a
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
movie_df=pd.read_csv('/content/drive/MyDrive/ml-latest-small (2)/ml-latest-small/movies.csv')
rating_df=pd.read_csv('/content/drive/MyDrive/ml-latest-small (2)/ml-latest-small/ratings.csv')

#movie:https://drive.google.com/file/d/1Zkw1uZkDFALG5c3_0-5fDWzGzrhAgw2_/view?usp=sharing
#rating:https://drive.google.com/file/d/1vaXI_sajmcchGfEYw6amEZyvVlC9MzqJ/view?usp=sharing

In [20]:
test_subject = 611
new_user={'userId':[test_subject,test_subject,test_subject,test_subject,test_subject],'movieId':[1,6,18,42,67],"rating":[5,4.5,3.5,5,7]}
new_user=pd.DataFrame(new_user)
df=pd.concat([rating_df,new_user])

In [21]:
rating_df.drop('timestamp',axis=1,inplace=True)

In [22]:
movieID_to_name = {}
with open('/content/drive/MyDrive/ml-latest-small (2)/ml-latest-small/movies.csv', newline='', encoding='ISO-8859-1') as csvfile:
    movie_reader = csv.reader(csvfile)
    next(movie_reader)
    for row in movie_reader:
        movieID = int(row[0])
        movie_name = row[1]
        movieID_to_name[movieID] = movie_name

In [23]:
from surprise import SVD,Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
reader = Reader()
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
algo = SVD()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8772  0.8643  0.8768  0.8811  0.8657  0.8730  0.0067  
MAE (testset)     0.6728  0.6646  0.6734  0.6760  0.6674  0.6708  0.0042  
Fit time          5.55    5.37    5.42    5.25    5.34    5.38    0.10    
Test time         0.29    0.27    0.16    0.26    0.31    0.26    0.05    


{'fit_time': (5.546561241149902,
  5.369401454925537,
  5.420727014541626,
  5.251547813415527,
  5.3365318775177),
 'test_mae': array([0.67276673, 0.66464727, 0.67338179, 0.67600547, 0.66735013]),
 'test_rmse': array([0.87718619, 0.86425392, 0.87676162, 0.88107486, 0.8657091 ]),
 'test_time': (0.28760695457458496,
  0.2676258087158203,
  0.16195154190063477,
  0.26464104652404785,
  0.3111400604248047)}

In [24]:
trainset = data.build_full_trainset()
similarity_matrix = algo.fit(trainset).compute_similarities()

In [25]:
test_subject_iid = trainset.to_inner_uid(test_subject)

# Get the top K items we rated
test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbors = heapq.nlargest(20, test_subject_ratings, key=lambda t: t[1])

In [26]:
candidates = defaultdict(float)

for itemID, rating in k_neighbors:
    try:
      similaritities = similarity_matrix[itemID]
      for innerID, score in enumerate(similaritities):
          candidates[innerID] += score * (rating / 5.0)
    except:
      continue

In [27]:

def getMovieName(movieID):
  if int(movieID) in movieID_to_name:
    return movieID_to_name[int(movieID)]
  else:
      return ""

In [28]:
watched = {}
for itemID, rating in trainset.ur[test_subject_iid]:
  watched[itemID] = 1

# Add items to list of user's recommendations

recommendations = []

position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
  if not itemID in watched:
    recommendations.append(getMovieName(trainset.to_raw_iid(itemID)))
    position += 1
    if (position > 10): break 

for rec in recommendations:
  print("Movie: ", rec)

Movie:  Bonnie and Clyde (1967)
Movie:  Safe (1995)
Movie:  Man Who Wasn't There, The (2001)
Movie:  Legend of Rita, The (Stille nach dem SchuÃ, Die) (1999)
Movie:  Scream 3 (2000)
Movie:  Best in Show (2000)
Movie:  Goodfellas (1990)
Movie:  Groundhog Day (1993)
Movie:  First Knight (1995)
Movie:  Talladega Nights: The Ballad of Ricky Bobby (2006)
Movie:  Dunston Checks In (1996)
